In [2]:
import numpy as np
subst_mat = np.array([
        [10,2,5,2],
        [2,10,2,5],
        [5,2,10,2],
        [2,5,2,10]
    ])

In [3]:
subst_mat[0,2]

5

In [4]:
from Bio import SeqIO

def read_FASTA(filename):
    records_dict = {}
    for seq_record in SeqIO.parse(filename, "fasta"):
        records_dict[seq_record.id] = seq_record.seq        
    return records_dict

seq1 = list(read_FASTA("seq1.fasta").values())[0]
seq2 = list(read_FASTA("seq2.fasta").values())[0]
print(seq1)
print(seq2)

GGCCTAAAGGCGCCGGTCTTTCGTACCCCAAAATCTCGGCATTTTAAGATAAGTGAGTGTTGCGTTACACTAGCGATCTACCGCGTCTTATACTTAAGCGTATGCCCAGATCTGACTAATCGTGCCCCCGGATTAGACGGGCTTGATGGGAAAGAACAGCTCGTCTGTTTACGTATAAACAGAATCGCCTGGGTTCGC
GGGCTAAAGGTTAGGGTCTTTCACACTAAAGAGTGGTGCGTATCGTGGCTAATGTACCGCTTCTGGTATCGTGGCTTACGGCCAGACCTACAAGTACTAGACCTGAGAACTAATCTTGTCGAGCCTTCCATTGAGGGTAATGGGAGAGAACATCGAGTCAGAAGTTATTCTTGTTTACGTAGAATCGCCTGGGTCCGC


In [5]:
T = np.full([len(seq1), len(seq2)],None)
def C(i,j):
    dict_subst = {"A":0, "C": 1, "G":2, "T":3}
    if T[i,j]==None : 
        v1,v2,v3,v4 = 0,0,0,0
        if i>0 and j>0 : 
            v1 = C(i-1,j-1)+ subst_mat[dict_subst[seq1[i-1]], dict_subst[seq2[j-1]]]
        if i>0 and j>=0 : 
            v2 = C(i-1,j)-5
        if i>=0 and j>0 : 
            v3 = C(i, j-1)-5
        if i==0 and j==0 : 
            v4 = 0
        T[i,j] = max(v1,v2,v3,v4)
    return T[i,j]

In [6]:
seq1 = "CGTGTCAAGTCT"
seq2 = "ACGTCGTAGCTAGG"
T = np.full([len(seq1)+1, len(seq2)+1],None)
C(len(seq1),len(seq2))

66

In [7]:
seq1 = list(read_FASTA("seq1.fasta").values())[0]
seq2 = list(read_FASTA("seq2.fasta").values())[0]
T = np.full([len(seq1)+1, len(seq2)+1],None)
mat=C(len(seq1),len(seq2))

In [7]:
T

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 10, 10, ..., 2, 10, 5],
       [0, 10, 20, ..., 5, 12, 12],
       ...,
       [0, 2, 5, ..., 1326, 1321, 1316],
       [0, 10, 12, ..., 1321, 1336, 1331],
       [0, 5, 12, ..., 1321, 1331, 1346]], dtype=object)

In [8]:
def backtrack(mat, n, m):
    if n==1 and m==1:
        return B
    elif n==1 :
        for i in range(m-1):
            B[1,m-i]="left"
        return(B)
    elif m==1 : 
        for i in range(n-1):
            B[n-i,1]="up"
        return B
    else :
        next_step = min(mat[n-1,m-1], mat[n-1,m], mat[n,m-1])
        if mat[n-1,m-1]==next_step :
            B[n,m]="diag"
            return backtrack(mat,n-1,m-1)
        if mat[n-1,m]==next_step :
            B[n,m]="up"
            return backtrack(mat,n-1,m)
        if mat[n,m-1]==next_step :
            B[n,m]="left"
            return backtrack(mat,n,m-1)

In [53]:
seq1 = "AATAAT" # len = 6
seq2 = "AAGG" # len = 4
T = np.full([len(seq1)+1, len(seq2)+1],None) # shape = (7,5)
mat = C(len(seq1),len(seq2))
print(mat, T)
B = np.full([len(seq1)+1, len(seq2)+1],"None")
B[1,1]= "Start"
mat = backtrack(T, T.shape[0]-1, T.shape[1]-1)

22 [[0 0 0 0 0]
 [0 10 10 5 5]
 [0 10 20 15 10]
 [0 5 15 22 17]
 [0 10 15 20 27]
 [0 10 20 20 25]
 [0 5 15 22 22]]


In [60]:
def read_align (mat):
    print(mat)
    n,m=mat.shape[0]-1, mat.shape[1]-1
    i,j=1,1
    align1=""
    align2=""
    while (i+1,j+1)!=(n,m):
        print(i,j)
        if mat[i+2,j+2]=="diag":
            print("here1")
            align1+=seq1[i-1]
            align2+=seq2[j-1]
            i+=1
            j+=1
        elif mat[i,j+1]=="left":
            print("here2")
            align1+="_"
            align2 += seq2[m-1]
            j+=1
        elif mat[i+1,j]=="up":
            print("here3")
            align1+= seq1[i-1]
            align2+="_"
            i+=1
    return (align1, align2)

In [61]:
read_align(mat)

[['None' 'None' 'None' 'None' 'None']
 ['None' 'Star' 'None' 'None' 'None']
 ['None' 'up' 'None' 'None' 'None']
 ['None' 'up' 'None' 'None' 'None']
 ['None' 'None' 'diag' 'None' 'None']
 ['None' 'None' 'None' 'diag' 'None']
 ['None' 'None' 'None' 'None' 'diag']]
1 1
here3
2 1
here3
3 1
here1
4 2
here1


('AATA', '__AA')

In [52]:
mat

array([[None, None, None, None, None],
       [None, 'Start', None, None, None],
       [None, 'up', None, None, None],
       [None, 'up', None, None, None],
       [None, None, 'diag', None, None],
       [None, None, None, 'diag', None],
       [None, None, None, None, 'diag']], dtype=object)

In [76]:
def RecurBackTrack(i,j, seq1, seq2, gap_cost, T, align1="", align2=""):
    dict_subst = {"A":0, "C": 1, "G":2, "T":3}
    print(i,j)
    print("cost",T[i,j], T[i-1,j]+ gap_cost )
    if i>0 and j>0 and T[i,j]==T[i-1,j-1]+ subst_mat[dict_subst[seq1[i-1]], dict_subst[seq2[j-1]]] :
        print("in 1")
        align1 = seq1[i-1] + align1
        align2 = seq2[j-1] + align2
        RecurBackTrack(i-1,j-1,seq1, seq2, gap_cost, T, align1, align2)
    elif i>0 and j>=0 and T[i,j]==T[i-1,j]+ gap_cost :
        print("in 2")
        align1 = seq1[i-1] + align1
        align2 = "_" + align2
        RecurBackTrack(i-1,j,seq1, seq2, gap_cost, T, align1, align2)
    elif i>=0 and j>0 and T[i,j]==T[i,j-1]+gap_cost : 
        print ("in 3")
        align1 = "_" + align1
        align2 = seq2[j-1] + align2
        RecurBackTrack(i,j-1, seq1, seq2, gap_cost, T, align1, align2)
    else:
        print( "in 4")
        print(align1, align2)
        return "hey"

In [72]:
def RecurBackTrack(i,j, seq1, seq2, gap_cost, T, align1="", align2=""):
    dict_subst = {"A":0, "C": 1, "G":2, "T":3}
    print(i,j)
    if i>0 and j>0 and T[i,j]==T[i-1,j-1]+ subst_mat[dict_subst[seq1[i-1]], dict_subst[seq2[j-1]]] :
        print("in 1")
        align1 = seq1[i-1] + align1
        align2 = seq2[j-1] + align2
        RecurBackTrack(i-1,j-1,seq1, seq2, gap_cost, T, align1, align2)
    elif i>0 and j>=0:
        print("in 2")
        align1 = seq1[i-1] + align1
        align2 = "_" + align2
        RecurBackTrack(i-1,j,seq1, seq2, gap_cost, T, align1, align2)
    elif i>=0 and j>0: 
        print ("in 3")
        align1 = "_" + align1
        align2 = seq2[j-1] + align2
        RecurBackTrack(i,j-1, seq1, seq2, gap_cost, T, align1, align2)
    else:
        print( "in 4")
        print(align1, align2)
        return align1

In [77]:
seq1 = "CGTGTCAAGTCT" 
seq2 = "ACGTCGTAGCTAGG" 
T = np.full([len(seq1)+1, len(seq2)+1],None)
mat = C(len(seq1),len(seq2))
RecurBackTrack(len(seq1),len(seq2), seq1, seq2, -5, T)
print(mat)

12 14
cost 66 61
in 1
11 13
cost 64 59
in 1
10 12
cost 62 49
in 3
10 11
cost 67 51
in 1
9 10
cost 57 49
in 1
8 9
cost 55 47
in 1
7 8
cost 50 42
in 1
6 7
cost 40 40
in 2
5 7
cost 45 25
in 1
4 6
cost 35 15
in 1
3 5
cost 25 5
in 3
3 4
cost 30 10
in 1
2 3
cost 20 0
in 1
1 2
cost 10 -5
in 1
0 1
cost 0 -3
in 4
CGT_GTCAAGT_CT CGTCGT_AGCTAGG
66


In [74]:
seq1 = "AATAAT" 
seq2 = "AAGG" 
T = np.full([len(seq1)+1, len(seq2)+1],None)
mat = C(len(seq1),len(seq2))
RecurBackTrack(len(seq1),len(seq2), seq1, seq2, -5, T)
print(mat)

6 4
in 1
5 3
in 1
4 2
in 1
3 1
in 2
2 1
in 1
1 0
in 2
0 0
in 4
AATAAT _A_AGG
22


In [65]:
T

array([[0, 0, 0, 0, 0, 0],
       [0, 10, 5, 2, 2, 10],
       [0, 5, 20, 15, 10, 7],
       [0, 5, 15, 22, 17, 15],
       [0, 2, 10, 20, 32, 27],
       [0, 2, 5, 20, 27, 34],
       [0, 2, 4, 15, 30, 29],
       [0, 2, 4, 14, 25, 32],
       [0, 2, 4, 9, 24, 27]], dtype=object)